In [1]:
# Install required packages.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install sentence-transformers
import torch
import pandas as pd
from torch_geometric.data import HeteroData, download_url, extract_zip
from torch_geometric.transforms import ToUndirected, RandomLinkSplit

from sklearn import preprocessing
import torch
import pandas as pd
import numpy as np
# from torch_geometric.data import Data
import os.path as osp
import inspect
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.nn import Parameter

# Helper functions for visualization.
%matplotlib inline

clinical = pd.read_csv('data/clinical_data_breast_cancer.csv', index_col=0)
proteomes_orig = pd.read_csv('data/77_cancer_proteomes_CPTAC_itraq.csv')
PAM50 = pd.read_csv('data/PAM50_proteins.csv')
proteomes = proteomes_orig.drop(['gene_symbol','gene_name'], axis=1)
clinical.index = clinical.index.to_series().apply(lambda title : title.split('CGA-')[1])
proteomes.rename(columns = proteomes.columns.to_series().apply(lambda title: title.split('.')[0]), inplace=True)

#Transpose and organize proteomes data
proteomes = proteomes.transpose()
proteomes.columns =  proteomes.iloc[0]
proteomes.drop('RefSeq_accession_number', axis=0, inplace=True)

#Convert gender to numbers
def num_gender(gender):
    if gender == 'MALE':
        return 0
    elif gender == 'FEMALE':
        return 1
    else:
        return float('NaN')

clinical['Gender'] = clinical['Gender'].apply(lambda gender: num_gender(gender))

#Convert status to numbers
def num_status(status):
    if status == 'Negative':
        return 0
    elif status == 'Positive':
        return 1
    else:
        return

clinical['ER Status'] = clinical['ER Status'].apply(lambda status: num_status(status))
clinical['PR Status'] = clinical['PR Status'].apply(lambda status: num_status(status))
clinical['HER2 Final Status'] = clinical['HER2 Final Status'].apply(lambda status: num_status(status))

#Convert tumor, node, metastasis to numbers
clinical['Tumor'] = clinical['Tumor'].apply(lambda tumor: tumor.split('T')[1])
clinical['Node'] = clinical['Node'].apply(lambda tumor: tumor.split('N')[1])
clinical['Metastasis'] = clinical['Metastasis'].apply(lambda tumor: tumor.split('M')[1])
#Remove unused columns
clinical.drop('Tumor--T1 Coded', axis=1, inplace=True)
clinical.drop('Metastasis-Coded', axis=1, inplace=True)
clinical.drop('Node-Coded', axis=1, inplace=True)

#Merge clinical data with proteomes data
datasetPrim = clinical.merge(proteomes, left_index=True,right_index=True)

datasetPrim

You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.


,Gender,Age at Initial Pathologic Diagnosis,ER Status,PR Status,HER2 Final Status,Tumor,Node,Metastasis,AJCC Stage,Converted Stage,...,NP_001193600,NP_061134,NP_932347,NP_003593,NP_997203,NP_001191293,NP_775791,NP_004065,NP_068752,NP_219494
A2-A0CM,1,40,0.0,0,0.0,2,0,0,Stage IIA,Stage IIA,...,NaN,NaN,1.153614,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2-A0D2,1,45,0.0,0,0.0,2,0,0,Stage IIB,Stage IIA,...,0.919136,-1.648856,0.832649,NaN,-8.324969,-4.679219,NaN,-1.10665,NaN,-6.941181
A2-A0EQ,1,64,0.0,0,1.0,2,0,0,Stage IIA,Stage IIA,...,-0.801685,NaN,NaN,3.80231,-6.373934,-1.12316,NaN,NaN,NaN,NaN
A2-A0EV,1,80,1.0,1,0.0,1,0,0,Stage IA,Stage I,...,-4.966177,-1.471027,NaN,-0.474013,-12.278546,-10.337729,-0.653251,NaN,NaN,NaN
A2-A0EX,1,46,1.0,1,0.0,3,0,0,Stage IIB,Stage IIB,...,1.45149,-2.018981,0.877456,NaN,NaN,-6.101005,NaN,-1.726336,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C8-A138,1,54,1.0,0,1.0,2,2,0,Stage III,Stage IIIA,...,-3.250913,1.711825,-0.248402,NaN,4.707022,4.107251,-3.20337,1.971481,NaN,NaN
D8-A142,1,74,0.0,0,0.0,3,0,0,Stage IIB,Stage IIB,...,-5.107629,-0.97598,NaN,2.508629,-12.33711,-9.54653,-4.066584,NaN,NaN,NaN
E2-A154,1,68,1.0,1,0.0,1,0,0,Stage I,Stage I,...,-3.386203,-2.328692,-2.806642,NaN,-4.733495,-9.584499,-4.786183,-3.103949,NaN,NaN
E2-A158,1,43,0.0,0,0.0,1,1,0,Stage IIA,Stage IIA,...,-0.638364,0.051811,2.509998,7.067839,NaN,0.378972,NaN,NaN,0.665797,NaN


In [2]:
noNaProteasomesColumns= np.array(datasetPrim.dropna(axis=1).columns[24:])
columnsOfIntrest = np.insert(noNaProteasomesColumns, 0, 'Metastasis', axis=0)
dataset=datasetPrim[columnsOfIntrest]

In [3]:
datasetMini = dataset.iloc[1:5,1:5]
datasetMini

,NP_958782,NP_958785,NP_958786,NP_000436
A2-A0D2,0.107491,0.104164,0.107491,0.097512
A2-A0EQ,-0.91267,-0.927979,-0.927979,-0.931806
A2-A0EV,0.452986,0.47259,0.47259,0.458587
A2-A0EX,1.185108,1.192612,1.18886,1.185108


In [4]:
import pandas as pd
import os.path as osp
import inspect
from torch_geometric.data import Data
from sklearn import preprocessing

import torch
import random
import numpy as np
import pdb

#seems that it will give some random masking of the nodes
def get_known_mask(known_prob, edge_num):
    known_mask = (torch.FloatTensor(edge_num, 1).uniform_() < known_prob).view(-1)
    return known_mask
#this seems to be mechanism for dropout of the nodes
def mask_edge(edge_index,edge_attr,mask,remove_edge):
    edge_index = edge_index.clone().detach()
    edge_attr = edge_attr.clone().detach()
    if remove_edge:
        edge_index = edge_index[:,mask]
        edge_attr = edge_attr[mask]
    else:
        edge_attr[~mask] = 0.
    return edge_index, edge_attr

#basically we get like adjacency matrix
def create_node(df, mode):
    if mode == 0: # onehot feature node, all 1 sample node
        nrow, ncol = df.shape
        feature_ind = np.array(range(ncol))
        feature_node = np.zeros((ncol,ncol))
        feature_node[np.arange(ncol), feature_ind] = 1
        sample_node = [[1]*ncol for i in range(nrow)]
        node = sample_node + feature_node.tolist()
    elif mode == 1: # onehot sample and feature node
        nrow, ncol = df.shape
        feature_ind = np.array(range(ncol))
        feature_node = np.zeros((ncol,ncol+1))
        feature_node[np.arange(ncol), feature_ind+1] = 1
        sample_node = np.zeros((nrow,ncol+1))
        sample_node[:,0] = 1
        node = sample_node.tolist() + feature_node.tolist()
    return node
#we get edges basically between all nodes on the left relative to the right so it will be bipartitie graph !!
def create_edge(df):
    n_row, n_col = df.shape
    edge_start = []
    edge_end = []
    for x in range(n_row):
        edge_start = edge_start + [x] * n_col # obj
        edge_end = edge_end + list(n_row+np.arange(n_col)) # att
    edge_start_new = edge_start + edge_end
    edge_end_new = edge_end + edge_start
    return (edge_start_new, edge_end_new)
#the entry in the table is saved as the edge attribute we are doubling its becouse we have bidirectional edge
def create_edge_attr(df):
    nrow, ncol = df.shape
    edge_attr = []
    for i in range(nrow):
        for j in range(ncol):
            edge_attr.append([float(df.iloc[i,j])])
    edge_attr = edge_attr + edge_attr
    return edge_attr

In [ ]:
#complete function to load data
def get_data(df_X, df_y, node_mode, train_edge_prob, split_sample_ratio, split_by, train_y_prob, seed=0, normalize=True):
    #I suppose just corner cases for small dataframes
    if len(df_y.shape)==1:
        df_y = df_y.to_numpy()
    elif len(df_y.shape)==2:
        df_y = df_y[0].to_numpy()
    #simple normalization of features as we see we ignore labels
    if normalize:
        x = df_X.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_X = pd.DataFrame(x_scaled)


    edge_start, edge_end = create_edge(df_X)
    edge_index = torch.tensor([edge_start, edge_end], dtype=int)
    edge_attr = torch.tensor(create_edge_attr(df_X), dtype=torch.float)
    node_init = create_node(df_X, node_mode)
    x = torch.tensor(node_init, dtype=torch.float)
    y = torch.tensor(df_y, dtype=torch.float)

    #set seed to fix known/unknwon edges
    torch.manual_seed(seed)
    #keep train_edge_prob of all edges
    train_edge_mask = get_known_mask(train_edge_prob, int(edge_attr.shape[0]/2))
    double_train_edge_mask = torch.cat((train_edge_mask, train_edge_mask), dim=0)

    #mask edges based on the generated train_edge_mask
    #train_edge_index is known, test_edge_index in unknwon, i.e. missing
    train_edge_index, train_edge_attr = mask_edge(edge_index, edge_attr,
                                                double_train_edge_mask, True)
    train_labels = train_edge_attr[:int(train_edge_attr.shape[0]/2),0]
    test_edge_index, test_edge_attr = mask_edge(edge_index, edge_attr,
                                                ~double_train_edge_mask, True)
    test_labels = test_edge_attr[:int(test_edge_attr.shape[0]/2),0]
    #mask the y-values during training, i.e. how we split the training and test sets
    train_y_mask = get_known_mask(train_y_prob, y.shape[0])
    test_y_mask = ~train_y_mask

    data = Data(x=x, y=y, edge_index=edge_index, edge_attr=edge_attr,
            train_y_mask=train_y_mask, test_y_mask=test_y_mask,
            train_edge_index=train_edge_index,train_edge_attr=train_edge_attr,
            train_edge_mask=train_edge_mask,train_labels=train_labels,
            test_edge_index=test_edge_index,test_edge_attr=test_edge_attr,
            test_edge_mask=~train_edge_mask,test_labels=test_labels,
            df_X=df_X,df_y=df_y,
            edge_attr_dim=train_edge_attr.shape[-1],
            user_num=df_X.shape[0]
            )

    if split_sample_ratio > 0.:
        if split_by == 'y':
            sorted_y, sorted_y_index = torch.sort(torch.reshape(y,(-1,)))
        elif split_by == 'random':
            sorted_y_index = torch.randperm(y.shape[0])
        lower_y_index = sorted_y_index[:int(np.floor(y.shape[0]*split_sample_ratio))]
        higher_y_index = sorted_y_index[int(np.floor(y.shape[0]*split_sample_ratio)):]
        # here we don't split x, only split edge
        # train
        half_train_edge_index = train_edge_index[:,:int(train_edge_index.shape[1]/2)];
        lower_train_edge_mask = []
        for node_index in half_train_edge_index[0]:
            if node_index in lower_y_index:
                lower_train_edge_mask.append(True)
            else:
                lower_train_edge_mask.append(False)
        lower_train_edge_mask = torch.tensor(lower_train_edge_mask)
        double_lower_train_edge_mask = torch.cat((lower_train_edge_mask, lower_train_edge_mask), dim=0)
        lower_train_edge_index, lower_train_edge_attr = mask_edge(train_edge_index, train_edge_attr,
                                                double_lower_train_edge_mask, True)
        lower_train_labels = lower_train_edge_attr[:int(lower_train_edge_attr.shape[0]/2),0]
        higher_train_edge_index, higher_train_edge_attr = mask_edge(train_edge_index, train_edge_attr,
                                                ~double_lower_train_edge_mask, True)
        higher_train_labels = higher_train_edge_attr[:int(higher_train_edge_attr.shape[0]/2),0]
        # test
        half_test_edge_index = test_edge_index[:,:int(test_edge_index.shape[1]/2)];
        lower_test_edge_mask = []
        for node_index in half_test_edge_index[0]:
            if node_index in lower_y_index:
                lower_test_edge_mask.append(True)
            else:
                lower_test_edge_mask.append(False)
        lower_test_edge_mask = torch.tensor(lower_test_edge_mask)
        double_lower_test_edge_mask = torch.cat((lower_test_edge_mask, lower_test_edge_mask), dim=0)
        lower_test_edge_index, lower_test_edge_attr = mask_edge(test_edge_index, test_edge_attr,
                                                double_lower_test_edge_mask, True)
        lower_test_labels = lower_test_edge_attr[:int(lower_test_edge_attr.shape[0]/2),0]
        higher_test_edge_index, higher_test_edge_attr = mask_edge(test_edge_index, test_edge_attr,
                                                ~double_lower_test_edge_mask, True)
        higher_test_labels = higher_test_edge_attr[:int(higher_test_edge_attr.shape[0]/2),0]


        data.lower_y_index = lower_y_index
        data.higher_y_index = higher_y_index
        data.lower_train_edge_index = lower_train_edge_index
        data.lower_train_edge_attr = lower_train_edge_attr
        data.lower_train_labels = lower_train_labels
        data.higher_train_edge_index = higher_train_edge_index
        data.higher_train_edge_attr = higher_train_edge_attr
        data.higher_train_labels = higher_train_labels
        data.lower_test_edge_index = lower_test_edge_index
        data.lower_test_edge_attr = lower_test_edge_attr
        data.lower_test_labels = lower_train_labels
        data.higher_test_edge_index = higher_test_edge_index
        data.higher_test_edge_attr = higher_test_edge_attr
        data.higher_test_labels = higher_test_labels

    return data
